In [ ]:
import numpy as np
import pylab as plt
import pandas as pd
import sklearn as sk
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils import shuffle
from sklearn.utils import resample

from matplotlib.patches import Circle, RegularPolygon
from matplotlib.path import Path
from matplotlib.projections import register_projection
from matplotlib.projections.polar import PolarAxes
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D

def balance_data(x,y):

    minor_index = int(y.mean()>0.5)
    minor_x = x[y==minor_index]
    major_x = x[y!=minor_index]

    minor_upsample = resample(minor_x,
                 replace=True,
                 n_samples=len(major_x),
                 random_state=42)
    upsampled_x = pd.concat([major_x, minor_upsample])
    upsampled_y = np.array(len(major_x)*[1-minor_index]+len(minor_upsample)*[minor_index])
    upsampled_x,upsampled_y = shuffle(upsampled_x,upsampled_y)
    return upsampled_x,upsampled_y

In [ ]:
df_base = pd.read_csv('sofa_parameters.csv').reset_index(names=['id'])

cols2 = [
    'id',
    'insurance',
    'marital_status',
    'race',
    'gender',
    'admission_age',
    'weight',
    'height',
    'charlson_comorbidity_index',
    '30d_mortality_status']
resp= ['respiration_SOFA_d1',
    'respiration_SOFA_d2',
    'respiration_SOFA_d3',
    'respiration_SOFA_d4',
    'respiration_SOFA_d5',
    'respiration_SOFA_d6',
    'respiration_SOFA_d7',
    'respiration_SOFA_d8',
    'respiration_SOFA_d9',
    'respiration_SOFA_d10',
    'respiration_SOFA_d11',
    'respiration_SOFA_d12',
    'respiration_SOFA_d13',
    'respiration_SOFA_d14']
coag =['coagulation_SOFA_d1',
    'coagulation_SOFA_d2',
    'coagulation_SOFA_d3',
    'coagulation_SOFA_d4',
    'coagulation_SOFA_d5',
    'coagulation_SOFA_d6',
    'coagulation_SOFA_d7',
    'coagulation_SOFA_d8',
    'coagulation_SOFA_d9',
    'coagulation_SOFA_d10',
    'coagulation_SOFA_d11',
    'coagulation_SOFA_d12',
    'coagulation_SOFA_d13',
    'coagulation_SOFA_d14']
liver= ['liver_SOFA_d1',
    'liver_SOFA_d2',
    'liver_SOFA_d3',
    'liver_SOFA_d4',
    'liver_SOFA_d5',
    'liver_SOFA_d6',
    'liver_SOFA_d7',
    'liver_SOFA_d8',
    'liver_SOFA_d9',
    'liver_SOFA_d10',
    'liver_SOFA_d11',
    'liver_SOFA_d12',
    'liver_SOFA_d13',
    'liver_SOFA_d14']
card=['cardiovascular_SOFA_d1',
    'cardiovascular_SOFA_d2',
    'cardiovascular_SOFA_d3',
    'cardiovascular_SOFA_d4',
    'cardiovascular_SOFA_d5',
    'cardiovascular_SOFA_d6',
    'cardiovascular_SOFA_d7',
    'cardiovascular_SOFA_d8',
    'cardiovascular_SOFA_d9',
    'cardiovascular_SOFA_d10',
    'cardiovascular_SOFA_d11',
    'cardiovascular_SOFA_d12',
    'cardiovascular_SOFA_d13',
    'cardiovascular_SOFA_d14']
cns=['cns_SOFA_d1',
    'cns_SOFA_d2',
    'cns_SOFA_d3',
    'cns_SOFA_d4',
    'cns_SOFA_d5',
    'cns_SOFA_d6',
    'cns_SOFA_d7',
    'cns_SOFA_d8',
    'cns_SOFA_d9',
    'cns_SOFA_d10',
    'cns_SOFA_d11',
    'cns_SOFA_d12',
    'cns_SOFA_d13',
    'cns_SOFA_d14']
renal=['renal_SOFA_d1',
    'renal_SOFA_d2',
    'renal_SOFA_d3',
    'renal_SOFA_d4',
    'renal_SOFA_d5',
    'renal_SOFA_d6',
    'renal_SOFA_d7',
    'renal_SOFA_d8',
    'renal_SOFA_d9',
    'renal_SOFA_d10',
    'renal_SOFA_d11',
    'renal_SOFA_d12',
    'renal_SOFA_d13',
    'renal_SOFA_d14']
sofa_ds = [
    'SOFA_d1',
    'SOFA_d2',
    'SOFA_d3',
    'SOFA_d4',
    'SOFA_d5',
    'SOFA_d6',
    'SOFA_d7',
    'SOFA_d8',
    'SOFA_d9',
    'SOFA_d10',
    'SOFA_d11',
    'SOFA_d12',
    'SOFA_d13',
    'SOFA_d14'
]
delta=['delta1_2',
    'delta2_3',
    'delta3_4',
    'delta4_5',
    'delta5_6',
    'delta6_7',
    'delta7_8',
    'delta8_9',
    'delta9_10',
    'delta10_11',
    'delta11_12',
    'delta12_13',
    'delta13_14']
auc=['Area1_2',
    'Area2_3',
    'Area3_4',
    'Area4_5',
    'Area5_6',
    'Area6_7',
    'Area7_8',
    'Area8_9',
    'Area9_10',
    'Area10_11',
    'Area11_12',
    'Area12_13',
    'Area13_14']

def build_set(val,n):
    n=n-2
    if n % 2 == 0:
        buffer_pre = (['┌']+['│']*int(n/2-1))
        buffer_post = (['│']*int(n/2)+['└'])
    else:
        buffer_pre = (['┌']+['│']*int((n-1)/2))
        buffer_post = (['│']*int((n-1)/2)+['└'])
    return buffer_pre,buffer_post

accs = {}
baccs = {}
f1s = {}
prs = {}
recs = {}
rocs = {}
cms = {}
count = -1

In [ ]:
for n in [3,8]:
    count = count + 1
    sofa_cols = sofa_ds[0:n+1]
    param_list =[*resp[0:n+1], *coag[0:n+1], *liver[0:n+1], *card[0:n+1], *cns[0:n+1],
                 *renal[0:n+1], *sofa_cols, *delta[0:n], *auc[0:n]]
    
    df_dofas = df_base[['id',*sofa_cols]].copy()
    df_dofas = df_dofas.dropna(how='all', subset = sofa_cols) 
    
    df = df_base[[
        'id',*cols2,*param_list
    ]].copy()
    
    df.loc[:,'TotalAUC'] = df[auc[0:n]].sum(axis=1)
    df.loc[:,'SOFA_max'] = df_dofas[sofa_cols].max(axis=1)
    df.loc[:,'SOFA_min'] = df_dofas[sofa_cols].min(axis=1)
    def f(x):
        if pd.isna(x): return np.nan
        return int(x.replace('SOFA_d',''))
    df.loc[:,'SOFA_max_day'] = df_dofas[sofa_cols].idxmax(axis=1,skipna=True).apply(lambda x: f(x))
    df.loc[:,'SOFA_min_day'] = df_dofas[sofa_cols].idxmin(axis=1,skipna=True).apply(lambda x: f(x))
    
    df.loc[:,'SOFA_range'] = df['SOFA_max_day']-df['SOFA_min_day']
    df.loc[:,'SOFA_long_stay'] = (~df_dofas[sofa_cols].isna()).sum(axis=1)

    merge_list = ['Respiration','Coagulation','Liver', 'Cardiovascular', 'CNS', 'Renal', 'SOFA']
    merge_list_calc = ['Delta', 'Sum']
    tick_list = cols2[1:-1]
    for item_list in merge_list:
        buffer_pre,buffer_post = build_set(item_list,n+1)
        tick_list = [*tick_list,*buffer_pre,item_list,*buffer_post]
    for item_list in merge_list_calc:
        buffer_pre,buffer_post = build_set(item_list,n)
        tick_list = [*tick_list,*buffer_pre,item_list,*buffer_post]

    tick_list=[*tick_list,'TotalAUC','SOFA_max','SOFA_min','SOFA_max_day','SOFA_min_day','SOFA_range','SOFA_long_stay']
    df = df.fillna(0).reset_index()
    
    le = {}
    for k in ['insurance','marital_status','race','gender']:
        le[k] = LabelEncoder()
        df[k] = le[k].fit_transform(df[k].astype(str))
    
    x = df.drop(columns=['id','id','index','30d_mortality_status'])
    y = df['30d_mortality_status']
    
    feat_set = list(x.columns)
    feat_dic = {j:i for i,j in enumerate(feat_set)}
    
    kf = KFold(n_splits=5)
    
    feat_list = []
    feature_importances_list = []
    accs[count] = []
    accs_eff = []
    baccs[count] = []
    f1s[count] = []
    prs[count] = []
    recs[count] = []
    rocs[count] = []
    cms[count] = []
    
    for i_try in range(5):
        for train_index, test_index in kf.split(x):
            x_train, x_test = x.iloc[train_index], x.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            x_train,y_train = balance_data(x_train,y_train)
            
            clf = RandomForestClassifier()
            clf.fit(x_train, y_train)
            y_pred = clf.predict(x_test)
            
            acc = accuracy_score(y_test,y_pred)
            bacc = balanced_accuracy_score(y_test,y_pred)
            f1 = f1_score(y_test,y_pred)
            pr = precision_score(y_test,y_pred)
            rec = recall_score(y_test,y_pred)
            roc = roc_auc_score(y_test,y_pred)
            cm = confusion_matrix(y_test, y_pred, labels=[0,1])
            
            accs[count].append(acc)
            accs_eff.append(acc)
            baccs[count].append(bacc)
            f1s[count].append(f1)
            prs[count].append(pr)
            recs[count].append(rec)
            rocs[count].append(roc)
            cms[count].append(cm)
            
            feat_list.extend(feat_set)
            feature_importances_list.append(clf.feature_importances_)
            accs_eff.append(clf.score(x_test,y_test))
    
    feat_x = [feat_dic[i] for i in feat_list]
    print(n, np.mean(accs_eff),np.std(accs_eff))
    disp = ConfusionMatrixDisplay(confusion_matrix=np.mean(np.array(cms[count]),axis=(0)).astype(int),display_labels=[0,1])
    disp.plot()
    plt.savefig('images/cm_all{}.tif'.format(n),dpi=300)
    ;
    
    fig,ax = plt.subplots(1,1,figsize=(18,6))
    ax.scatter(feat_x,feature_importances_list,marker='.')
    ax.set_ylabel('feature importances')
    ax.set_xticks(np.arange(len(feat_set)))
    ax.set_xticklabels(tick_list, rotation=90)
    ax.set_xlim(-0.5,len(feat_set)-0.5)
    
    title = ''
    title += f'acc={np.mean(accs_eff):.2f}({np.std(accs_eff):.2f}), '
    title += f'balanced acc={np.mean(baccs[count]):.2f}({np.std(baccs[count]):.2f}), '
    title += f'f1={np.mean(f1s[count]):.2f}({np.std(f1s[count]):.2f})\n'
    title += f'precision={np.mean(prs[count]):.2f}({np.std(prs[count]):.2f}), '
    title += f'recall={np.mean(recs[count]):.2f}({np.std(recs[count]):.2f}), '
    title += f'ROC={np.mean(rocs[count]):.2f}({np.std(rocs[count]):.2f})'
    
    ax.set_title(title)
    plt.subplots_adjust(bottom=0.36)
    plt.savefig('images/set_all{}.tif'.format(n),dpi=300)
    ;

In [ ]:
for n in [3,8]:
    count = count + 1
    sofa_cols = sofa_ds[0:n+1]
    param_list =sofa_cols
    
    df_dofas = df_base[['id',*sofa_cols]].copy()
    df_dofas = df_dofas.dropna(how='all', subset = sofa_cols) 
    
    df = df_base[[
        'id',*cols2,*param_list
    ]].copy()
    
    df.loc[:,'TotalAUC'] = df_base[auc[0:n]].sum(axis=1)
    df.loc[:,'SOFA_max'] = df_dofas[sofa_cols].max(axis=1)
    df.loc[:,'SOFA_min'] = df_dofas[sofa_cols].min(axis=1)
    def f(x):
        if pd.isna(x): return np.nan
        return int(x.replace('SOFA_d',''))
    df.loc[:,'SOFA_max_day'] = df_dofas[sofa_cols].idxmax(axis=1,skipna=True).apply(lambda x: f(x))
    df.loc[:,'SOFA_min_day'] = df_dofas[sofa_cols].idxmin(axis=1,skipna=True).apply(lambda x: f(x))
    
    df.loc[:,'SOFA_range'] = df['SOFA_max_day']-df['SOFA_min_day']
    df.loc[:,'SOFA_long_stay'] = (~df_dofas[sofa_cols].isna()).sum(axis=1)

    merge_list = ['SOFA']
    merge_list_calc = []
    tick_list = cols2[1:-1]
    for item_list in merge_list:
        buffer_pre,buffer_post = build_set(item_list,n+1)
        tick_list = [*tick_list,*buffer_pre,item_list,*buffer_post]
    for item_list in merge_list_calc:
        buffer_pre,buffer_post = build_set(item_list,n)
        tick_list = [*tick_list,*buffer_pre,item_list,*buffer_post]

    tick_list=[*tick_list,'TotalAUC','SOFA_max','SOFA_min','SOFA_max_day','SOFA_min_day','SOFA_range','SOFA_long_stay']
    df = df.fillna(0).reset_index()
    
    le = {}
    for k in ['insurance','marital_status','race','gender']:
        le[k] = LabelEncoder()
        df[k] = le[k].fit_transform(df[k].astype(str))
    
    x = df.drop(columns=['id','id','index','30d_mortality_status'])
    y = df['30d_mortality_status']
    
    feat_set = list(x.columns)
    feat_dic = {j:i for i,j in enumerate(feat_set)}
    
    kf = KFold(n_splits=5)
    
    feat_list = []
    feature_importances_list = []
    accs[count] = []
    accs_eff = []
    baccs[count] = []
    f1s[count] = []
    prs[count] = []
    recs[count] = []
    rocs[count] = []
    cms[count] = []
    
    for i_try in range(5):
        for train_index, test_index in kf.split(x):
            x_train, x_test = x.iloc[train_index], x.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            x_train,y_train = balance_data(x_train,y_train)
            
            clf = RandomForestClassifier()
            clf.fit(x_train, y_train)
            y_pred = clf.predict(x_test)
            
            acc = accuracy_score(y_test,y_pred)
            bacc = balanced_accuracy_score(y_test,y_pred)
            f1 = f1_score(y_test,y_pred)
            pr = precision_score(y_test,y_pred)
            rec = recall_score(y_test,y_pred)
            roc = roc_auc_score(y_test,y_pred)
            cm = confusion_matrix(y_test, y_pred, labels=[0,1])
            
            accs[count].append(acc)
            accs_eff.append(acc)
            baccs[count].append(bacc)
            f1s[count].append(f1)
            prs[count].append(pr)
            recs[count].append(rec)
            rocs[count].append(roc)
            cms[count].append(cm)
            
            feat_list.extend(feat_set)
            feature_importances_list.append(clf.feature_importances_)
            accs_eff.append(clf.score(x_test,y_test))
    
    feat_x = [feat_dic[i] for i in feat_list]
    print(n, np.mean(accs_eff),np.std(accs_eff))
    disp = ConfusionMatrixDisplay(confusion_matrix=np.mean(np.array(cms[count]),axis=(0)).astype(int),display_labels=[0,1])
    disp.plot()
    plt.savefig('images/cm_dem{}.tif'.format(n),dpi=300)
    ;
    
    fig,ax = plt.subplots(1,1,figsize=(18,6))
    ax.scatter(feat_x,feature_importances_list,marker='.')
    ax.set_ylabel('feature importances')
    ax.set_xticks(np.arange(len(feat_set)))
    ax.set_xticklabels(tick_list, rotation=90)
    ax.set_xlim(-0.5,len(feat_set)-0.5)
    
    title = ''
    title += f'acc={np.mean(accs_eff):.2f}({np.std(accs_eff):.2f}), '
    title += f'balanced acc={np.mean(baccs[count]):.2f}({np.std(baccs[count]):.2f}), '
    title += f'f1={np.mean(f1s[count]):.2f}({np.std(f1s[count]):.2f})\n'
    title += f'precision={np.mean(prs[count]):.2f}({np.std(prs[count]):.2f}), '
    title += f'recall={np.mean(recs[count]):.2f}({np.std(recs[count]):.2f}), '
    title += f'ROC={np.mean(rocs[count]):.2f}({np.std(rocs[count]):.2f})'
    
    ax.set_title(title)
    plt.subplots_adjust(bottom=0.36)
    plt.savefig('images/set_dem{}.tif'.format(n),dpi=300)
    ;

In [ ]:
for n in [3,8]:
    count = count + 1
    sofa_cols = sofa_ds[0:n+1]
    param_list =sofa_cols
    
    df_dofas = df_base[['id',*sofa_cols]].copy()
    df_dofas = df_dofas.dropna(how='all', subset = sofa_cols) 
    
    df = df_base[[
        'id',*cols2,*param_list
    ]].copy()
    
    df.loc[:,'TotalAUC'] = df_base[auc[0:n]].sum(axis=1)
    df.loc[:,'SOFA_max'] = df_dofas[sofa_cols].max(axis=1)
    df.loc[:,'SOFA_min'] = df_dofas[sofa_cols].min(axis=1)
    def f(x):
        if pd.isna(x): return np.nan
        return int(x.replace('SOFA_d',''))
    df.loc[:,'SOFA_max_day'] = df_dofas[sofa_cols].idxmax(axis=1,skipna=True).apply(lambda x: f(x))
    df.loc[:,'SOFA_min_day'] = df_dofas[sofa_cols].idxmin(axis=1,skipna=True).apply(lambda x: f(x))
    
    df.loc[:,'SOFA_range'] = df['SOFA_max_day']-df['SOFA_min_day']
    df.loc[:,'SOFA_long_stay'] = (~df_dofas[sofa_cols].isna()).sum(axis=1)

    merge_list = ['SOFA']
    merge_list_calc = []
    tick_list = []
    for item_list in merge_list:
        buffer_pre,buffer_post = build_set(item_list,n+1)
        tick_list = [*tick_list,*buffer_pre,item_list,*buffer_post]
    for item_list in merge_list_calc:
        buffer_pre,buffer_post = build_set(item_list,n)
        tick_list = [*tick_list,*buffer_pre,item_list,*buffer_post]

    tick_list=[*tick_list,'TotalAUC','SOFA_max','SOFA_min','SOFA_max_day','SOFA_min_day','SOFA_range','SOFA_long_stay']
    df = df.fillna(0).reset_index()

    x = df.drop(columns=['id','id', 'insurance', 'marital_status', 'race', 'gender',
       'admission_age', 'weight', 'height','charlson_comorbidity_index','index','30d_mortality_status'])
    y = df['30d_mortality_status']
    
    feat_set = list(x.columns)
    feat_dic = {j:i for i,j in enumerate(feat_set)}
    
    kf = KFold(n_splits=5)
    
    feat_list = []
    feature_importances_list = []
    accs[count] = []
    accs_eff = []
    baccs[count] = []
    f1s[count] = []
    prs[count] = []
    recs[count] = []
    rocs[count] = []
    cms[count] = []
    
    for i_try in range(5):
        for train_index, test_index in kf.split(x):
            x_train, x_test = x.iloc[train_index], x.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            x_train,y_train = balance_data(x_train,y_train)
            
            clf = RandomForestClassifier()
            clf.fit(x_train, y_train)
            y_pred = clf.predict(x_test)
            
            acc = accuracy_score(y_test,y_pred)
            bacc = balanced_accuracy_score(y_test,y_pred)
            f1 = f1_score(y_test,y_pred)
            pr = precision_score(y_test,y_pred)
            rec = recall_score(y_test,y_pred)
            roc = roc_auc_score(y_test,y_pred)
            cm = confusion_matrix(y_test, y_pred, labels=[0,1])
            
            accs[count].append(acc)
            accs_eff.append(acc)
            baccs[count].append(bacc)
            f1s[count].append(f1)
            prs[count].append(pr)
            recs[count].append(rec)
            rocs[count].append(roc)
            cms[count].append(cm)
            
            feat_list.extend(feat_set)
            feature_importances_list.append(clf.feature_importances_)
            accs_eff.append(clf.score(x_test,y_test))
    
    feat_x = [feat_dic[i] for i in feat_list]
    print(n, np.mean(accs_eff),np.std(accs_eff))
    disp = ConfusionMatrixDisplay(confusion_matrix=np.mean(np.array(cms[count]),axis=(0)).astype(int),display_labels=[0,1])
    disp.plot()
    plt.savefig('images/cm_sofa{}.tif'.format(n),dpi=300)
    ;
    
    fig,ax = plt.subplots(1,1,figsize=(18,6))
    ax.scatter(feat_x,feature_importances_list,marker='.')
    ax.set_ylabel('feature importances')
    ax.set_xticks(np.arange(len(feat_set)))
    ax.set_xticklabels(tick_list, rotation=90)
    ax.set_xlim(-0.5,len(feat_set)-0.5)
    
    title = ''
    title += f'acc={np.mean(accs_eff):.2f}({np.std(accs_eff):.2f}), '
    title += f'balanced acc={np.mean(baccs[count]):.2f}({np.std(baccs[count]):.2f}), '
    title += f'f1={np.mean(f1s[count]):.2f}({np.std(f1s[count]):.2f})\n'
    title += f'precision={np.mean(prs[count]):.2f}({np.std(prs[count]):.2f}), '
    title += f'recall={np.mean(recs[count]):.2f}({np.std(recs[count]):.2f}), '
    title += f'ROC={np.mean(rocs[count]):.2f}({np.std(rocs[count]):.2f})'
    
    ax.set_title(title)
    plt.subplots_adjust(bottom=0.36)
    plt.savefig('images/set_sofa{}.tif'.format(n),dpi=300)
    ;

# Spider plot

In [ ]:
metrics = []
set_list = ['3 days All','8 days All','3 days dem','8 days dem','3 days SOFA','8 days SOFA']
for k in range(6):
    #kk = np.stack([accs[k],baccs[k],f1s[k],prs[k],recs[k],rocs[k]],axis=0)
    cm3 = np.array(cms[k])
    tn = cm3[:,0,0]
    fp = cm3[:,0,1]
    fn = cm3[:,1,0]
    tp = cm3[:,1,1]
    npv = tn/(fn+tn)
    spec = 2*np.array(baccs[k]) - np.array(recs[k])
    kk = np.stack([recs[k],2*np.array(prs[k]),spec,npv,rocs[k]],axis=0)
    metrics.append(kk)
metrics = np.mean(np.stack(metrics,axis=0),axis=2)
#metrics.shape

In [ ]:
def radar_factory(num_vars, frame='circle'):
    """
    Create a radar chart with `num_vars` axes.

    This function creates a RadarAxes projection and registers it.

    Parameters
    ----------
    num_vars : int
        Number of variables for radar chart.
    frame : {'circle', 'polygon'}
        Shape of frame surrounding axes.

    """
    # calculate evenly-spaced axis angles
    theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False)

    class RadarTransform(PolarAxes.PolarTransform):

        def transform_path_non_affine(self, path):
            # Paths with non-unit interpolation steps correspond to gridlines,
            # in which case we force interpolation (to defeat PolarTransform's
            # autoconversion to circular arcs).
            if path._interpolation_steps > 1:
                path = path.interpolated(num_vars)
            return Path(self.transform(path.vertices), path.codes)

    class RadarAxes(PolarAxes):

        name = 'radar'
        PolarTransform = RadarTransform

        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            # rotate plot such that the first axis is at the top
            self.set_theta_zero_location('N')

        def fill(self, *args, closed=True, **kwargs):
            """Override fill so that line is closed by default"""
            return super().fill(closed=closed, *args, **kwargs)

        def plot(self, *args, **kwargs):
            """Override plot so that line is closed by default"""
            lines = super().plot(*args, **kwargs)
            for line in lines:
                self._close_line(line)

        def _close_line(self, line):
            x, y = line.get_data()
            # FIXME: markers at x[0], y[0] get doubled-up
            if x[0] != x[-1]:
                x = np.append(x, x[0])
                y = np.append(y, y[0])
                line.set_data(x, y)

        def set_varlabels(self, labels):
            self.set_thetagrids(np.degrees(theta), labels)

        def _gen_axes_patch(self):
            # The Axes patch must be centered at (0.5, 0.5) and of radius 0.5
            # in axes coordinates.
            if frame == 'circle':
                return Circle((0.5, 0.5), 0.5)
            elif frame == 'polygon':
                return RegularPolygon((0.5, 0.5), num_vars,
                                      radius=.5, edgecolor="k")
            else:
                raise ValueError("Unknown value for 'frame': %s" % frame)

        def _gen_axes_spines(self):
            if frame == 'circle':
                return super()._gen_axes_spines()
            elif frame == 'polygon':
                spine = Spine(axes=self,
                              spine_type='circle',
                              path=Path.unit_regular_polygon(num_vars))
                spine.set_transform(Affine2D().scale(.5).translate(.5, .5)
                                    + self.transAxes)
                return {'polar': spine}
            else:
                raise ValueError("Unknown value for 'frame': %s" % frame)

    register_projection(RadarAxes)
    return theta

# Precision = PPV
# Recall = Sensitivity
spoke_labels = ['Sensitivity','2xPPV',  'Specificity', 'NPV','ROC']

theta = radar_factory(len(spoke_labels), frame='polygon')

fig, ax = plt.subplots(figsize=(9, 5), nrows=1, ncols=1,
                        subplot_kw=dict(projection='radar'))
fig.subplots_adjust(wspace=0.25, hspace=0.20, top=0.85, bottom=0.05)

#colors = ['b', 'r', 'g', 'm', 'y','k']
colors = ['b', 'b', 'r','r', 'm','m']
linestyles = ['dotted','solid','dotted','solid','dotted','solid']
# Plot the six cases from the example data on separate axes

title = 'Model Comparison'
case_data = metrics

ax.set_rgrids([0.4, 0.5, 0.6, 0.7, 0.8], angle=90)
ax.set_title(title, weight='bold', size='medium', position=(0.5, 1.1),
             horizontalalignment='center', verticalalignment='center')
for d, color,linestyle in zip(case_data, colors,linestyles):
    ax.plot(theta, d, color=color, linestyle=linestyle)
    #ax.fill(theta, d, facecolor=color, alpha=0.25, label='_nolegend_')
ax.set_varlabels(spoke_labels)
ax.set_ylim([0.5,1])

# add legend relative to top-left plot
labels = set_list
legend = ax.legend(labels, loc=(0.9, .95),
                          labelspacing=0.1, fontsize='small')
plt.savefig('images/spider.tif',dpi=300)